In [1]:
!pip install Selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!apt-get install -y fonts-nanum*
!pip install requests

'apt-get'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'apt'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'apt-get'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [10]:
def scroll_down():
  body = webdriver.find_element_by_css_selector('body')
  while True:
    last_height = webdriver.execute_script("return document.body.scrollHeight")
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    webdriver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    new_height = webdriver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
      break
    last_height = new_height

In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import time
from collections import deque
import pandas as pd
from pandas import Series
import numpy as np
import requests

# 웹드라이버 옵션 추가
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--start-maximized")
# chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage') #, chrome_options=chrome_options)

webdriver = webdriver.Chrome('C:/Users/sunha/OneDrive/Desktop/github/chromedriver.exe', options=chrome_options)

# url 링크
url = 'https://www.wadiz.kr/web/wreward/category/312?keyword=&endYn=ALL&order=recommend'
webdriver.get(url)
body_main = webdriver.find_element_by_css_selector('body')

# 스크롤 다운 함수
scroll_down()

In [12]:
len(list_href)

675

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import time
from collections import deque
import pandas as pd
from pandas import Series
import numpy as np
import requests

# 웹드라이버 옵션 추가
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--start-maximized")
# chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage') #, chrome_options=chrome_options)

webdriver = webdriver.Chrome('C:/Users/sunha/OneDrive/Desktop/github/chromedriver.exe', options=chrome_options)

# url 링크
url = 'https://www.wadiz.kr/web/wreward/category/312?keyword=&endYn=ALL&order=recommend'
webdriver.get(url)
body_main = webdriver.find_element_by_css_selector('body')

# 스크롤 다운 함수
scroll_down()

titles, categories, company_names, percentages, funding_amounts, days_left = [],[],[],[],[],[]
n_supporters, likes = [],[]
durations, target_amounts = [],[]
descriptions = []

atag = webdriver.find_elements_by_css_selector("#main-app > div.MainWrapper_content__GZkTa > div > div.RewardProjectListApp_container__1ZYeD.RewardMainProjectList_listApp__2noRS > div.ProjectCardList_container__3Y14k > div.ProjectCardList_list__1YBa2 > div a")

list_href = set([a.get_attribute('href') for a in atag])

for href in list_href:  # len(titles) + 1
    url1 = requests.get(href).text
    webdriver.get(href)
    time.sleep(0.3)
    page_source = webdriver.page_source
#     soup = bs(url1, 'html.parser')
    soup = bs(page_source, 'html.parser')
    
    #categories
    category = soup.find('p',class_ = 'title-info').text
    categories.append(category)
    
    #titles
    title = soup.find('h2',class_ = 'title').text
    titles.append(title)
    
    #company_names
    company_name = soup.find('div', class_ ='RewardMakerInfoBox_makerName__3TOmT').text
    company_names.append(company_name)

    #percentages
    percentage = soup.find('p', class_ = 'achievement-rate').text.split(' ')[0]
    percentages.append(percentage)
    
    #funding_amounts
    funding = soup.find('p', class_ = 'total-amount').text.split(' ')[0]
    funding_amounts.append(funding)
    
    #days_left
    
    try: 
        day = soup.find('p', class_ = 'remaining-day').text
    except AttributeError as e:
        days_left.append(day)
    else:
        days_left.append('None')
        
    #n_supporters
    supporter = soup.find('p',class_ = 'total-supporter').text.split('명')[0]
    n_supporters.append(supporter)
    
    #likes
    like = soup.find('em',class_='cnt-like').text
    likes.append(like.strip())

    #durations,target_amounts
    info = soup.find('p',style='color:#00b2b2;font-size:13px;line-height:20px;margin-bottom:8px;').text.split('\n')
    duration = info[2].split(' ')[-1]
    durations.append(duration)
    target_amount = info[1].split(' ')[-1]
    target_amounts.append(target_amount)

    #descriptions
    description = soup.select("#container > div.reward-body-wrap > div > div.wd-ui-info-wrap > div.wd-ui-sub-campaign-info-container > div > div > section > div.campaign-summary")[0]
    descriptions.append(description.text.strip())

In [7]:
len(list_href)

675

In [8]:
data = {
    'categories':categories,
    'title':titles,
    "company_names":company_names,
    'percentages':percentages, 
    'funding_amounts' :funding_amounts}

df2 = pd.DataFrame(data)
df2['target_amounts'] = target_amounts
df2['n_supporters'] = n_supporters
df2['likes'] = likes
df2['descriptions'] = descriptions
df2['durations'] = durations
df2['days_left'] = days_left

In [9]:
df2.to_csv('donation.csv', encoding = 'utf-8')

In [47]:
days_left

['None',
 '펀딩성공',
 '펀딩성공',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 '펀딩성공',
 'None',
 '펀딩성공',
 'None',
 '펀딩성공',
 '펀딩성공',
 'None',
 'None',
 '펀딩성공',
 'None',
 'None',
 '펀딩성공',
 'None',
 '펀딩성공',
 'None',
 '펀딩성공',
 '펀딩성공',
 '펀딩성공',
 '펀딩성공',
 'None',
 '펀딩성공',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 '펀딩성공',
 'None',
 'None',
 'None',
 '펀딩성공',
 'None',
 '펀딩성공',
 'None',
 '펀딩성공',
 'None',
 '펀딩성공',
 'None',
 'None',
 '펀딩성공',
 '펀딩성공',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 'None',
 '펀딩성공',
 'None',
 'None',
 '15일 남음',
 'None',
 'None',
 'None',
 'None',
 '펀딩성공',
 'None',
 '펀딩성공',
 'None',
 'None',
 'None',
 '펀딩성공',
 'None',
 'None',
 '펀딩성공',
 'None',
 'None',
 '펀딩성공',
 'None',
 'None',
 '펀딩성공',
 'None',
 '펀딩성공',
 'None',
 'None',
 'None',
 '펀딩성공',
 'None',
 'None',
 'None',
 'None',
 '펀딩성공',
 '펀딩성공',
 'None',
 'None',
 '펀딩성공',
 'None',
 'None',
 '펀딩성공',
 'None',
 'None',
 '펀딩성공',
 'None',
 'None',
 'None',